In [1]:
import os
import pickle
from tqdm import tqdm
import torch
from token_utils_rep import EHRTokenizer
from dataset_utils_rep import HBERTPretrainEHRDataset, batcher
from torch.utils.data import DataLoader
from HEART_rep import HBERT_Pretrain
from set_seed_utils import set_random_seed

Disabling PyTorch because PyTorch >= 2.1 is required but found 1.13.1
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
args = {
    "seed": 0,
    "dataset": "MIMIC-III",  # MIMIC-III, MIMIC-IV
    "batch_size": 32,
    "lr": 2e-5,
    "epochs": 30,
    "encoder": "hi_edge",
    "mask_rate": 0.7,
    "anomaly_rate": 0.05,
    "anomaly_loss_weight": 1,
    "num_hidden_layers": 5,
    "num_attention_heads": 6,
    "attention_probs_dropout_prob": 0.2,
    "hidden_dropout_prob": 0.2,
    "edge_hidden_size": 32,
    "hidden_size": 288,  # must be divisible by num_attention_heads
    "intermediate_size": 288,
    "gnn_n_heads": 1,
    "gnn_temp": 1,
    "gat": "dotattn",  # dotattn, None
    "diag_med_emb": "tree",  # simple, tree
}

In [4]:
# here we only use the MIMIC dataset
args['max_visit_size'] = 15
args['predicted_token_type'] = ["diag", "med", "pro", "lab"]
args['mask_token_id'] = {"diag":3, "med":4, "pro":5, "lab":6}  # {token_type: masked_id}
args['special_tokens'] = ("[PAD]", "[CLS]", "[SEP]", 
                       "[MASK0]", "[MASK1]", "[MASK2]", "[MASK3]")
# note that here "[MASK0]", "[MASK1]", "[MASK2]", "[MASK3]" are used for masking pretraining task
# codes that are actually masked are not in the input sequence

In [5]:
full_data_path = f"/home/lideyi/HeteroGT-cuda/data_process/{args['dataset']}-processed/mimic.pkl"
pretrain_data_path = f"/home/lideyi/HeteroGT-cuda/data_process/{args['dataset']}-processed/mimic_pretrain.pkl" # for pretraining

In [6]:
ehr_data = pickle.load(open(full_data_path, 'rb'))

In [7]:
diag_sentences = ehr_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_data["NDC"].values.tolist()
# lab was cut in 5 percentiles
lab_sentences = ehr_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_data["PRO_CODE"].values.tolist()
gender_set = [["M"], ["F"]]
age_set = [[c] for c in set(ehr_data["AGE"].values.tolist())]
age_gender_set = [[str(c) + "_" + gender] \
    for c in set(ehr_data["AGE"].values.tolist()) for gender in ["M", "F"]]

In [8]:
# tokenizer used full data
tokenizer = EHRTokenizer(diag_sentences, med_sentences, 
                         lab_sentences, pro_sentences, gender_set, 
                         age_set, age_gender_set, special_tokens=args["special_tokens"])
tokenizer.build_tree()

In [9]:
ehr_pretrain_data = pickle.load(open(pretrain_data_path, 'rb'))
pretrain_dataset = HBERTPretrainEHRDataset(ehr_pretrain_data, tokenizer, 
                                  token_type=args['predicted_token_type'], 
                                  mask_rate=args['mask_rate'],
                                  anomaly_rate=args['anomaly_rate'])

In [10]:
pretrain_dataloader = DataLoader(pretrain_dataset, batch_size=args["batch_size"], 
                                 collate_fn=batcher(pad_id = tokenizer.vocab.word2id["[PAD]"], 
                                                    n_token_type=len(args["predicted_token_type"]), is_train = True),
                                 shuffle=True)

In [11]:
set_random_seed(args["seed"])

[INFO] Random seed set to 0


In [12]:
exp_name = "Pretrain-HBERT" \
    + "-" + str(args["dataset"]) \
    + "-" + str(args["encoder"]) \
    + "-" + str(args["mask_rate"]) \
    + "-" + str(args["anomaly_rate"]) \
    + "-" + str(args["anomaly_loss_weight"]) \
    + "-" + str(args["hidden_size"]) \
    + "-" + str(args["edge_hidden_size"]) \
    + "-" + str(args["num_hidden_layers"]) \
    + "-" + str(args["num_attention_heads"]) \
    + "-" + str(args["attention_probs_dropout_prob"]) \
    + "-" + str(args["hidden_dropout_prob"]) \
    + "-" + str(args["intermediate_size"]) \
    + "-" + str(args["gat"]) \
    + "-" + str(args["gnn_n_heads"]) \
    + "-" + str(args["gnn_temp"]) \
    + "-" + str(args["diag_med_emb"])
print(exp_name)

save_path = "./pretrained_models/" + exp_name
if not os.path.exists(save_path):
    os.makedirs(save_path)

Pretrain-HBERT-MIMIC-III-hi_edge-0.7-0.05-1-288-32-5-6-0.2-0.2-288-dotattn-1-1-tree


In [13]:
args["vocab_size"] = 7 + len(tokenizer.diag_voc.id2word) + \
                len(tokenizer.pro_voc.id2word) + \
                len(tokenizer.med_voc.id2word) + \
                len(tokenizer.lab_voc.id2word) + \
                len(tokenizer.age_voc.id2word) + \
                len(tokenizer.gender_voc.id2word) + \
                len(tokenizer.age_gender_voc.id2word)

args["label_vocab_size"] = {"diag":len(tokenizer.diag_voc.id2word), 
                        "pro":len(tokenizer.pro_voc.id2word), 
                        "med":len(tokenizer.med_voc.id2word), 
                        "lab":len(tokenizer.lab_voc.id2word)}  # {token_type: vocab_size}

In [14]:
loss_entity = ["diag", "med", "pro", "lab", "anomaly"]

In [15]:
model = HBERT_Pretrain(args, tokenizer).to(device)

In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=args["lr"])

In [17]:
for epoch in range(1, 1 + args["epochs"]):
    train_iter = tqdm(pretrain_dataloader, ncols=140)
    model.train()
    ave_loss, ave_loss_dict = 0., {token_type: 0. for token_type in loss_entity}

    for step, batch in enumerate(train_iter):

        batch = [x.to(device) if isinstance(x, torch.Tensor) else x for x in batch]
        loss, loss_dict, perf_dict = model(*batch)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_iter.set_description(f"Epoch:{epoch: 03d}, Step:{step: 03d}, loss:{loss.item():.4f}, diag:{loss_dict['diag']:.4f}, med:{loss_dict['med']:.4f}, pro:{loss_dict['pro']:.4f}, lab:{loss_dict['lab']:.4f}, anomaly:{loss_dict['anomaly']:.4f}")

        ave_loss += loss.item()
        ave_loss_dict = {token_type: ave_loss_dict[token_type] + loss_dict[token_type] for token_type in loss_entity}

    ave_loss /= (step + 1)
    ave_loss_dict = {token_type: ave_loss_dict[token_type] / (step + 1) for token_type in loss_entity}
    print(f"Epoch {epoch} finished, ave_loss: {ave_loss:.4f}, ave_loss_dict: {ave_loss_dict}, perf_dict: {perf_dict}")

Epoch: 01, Step: 723, loss:0.0757, diag:0.0160, med:0.0942, pro:0.0134, lab:0.0577, anomaly:0.1970: 100%|█| 724/724 [01:34<00:00,  7.70it/s]


Epoch 1 finished, ave_loss: 0.1581, ave_loss_dict: {'diag': 0.11248204618620362, 'med': 0.15466383776694373, 'pro': 0.11071781535730671, 'lab': 0.1528678211172559, 'anomaly': 0.25975676049679025}, perf_dict: {'diag': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'med': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'pro': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'lab': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}}


Epoch: 02, Step: 723, loss:0.0650, diag:0.0144, med:0.0704, pro:0.0136, lab:0.0670, anomaly:0.1594: 100%|█| 724/724 [01:30<00:00,  7.97it/s]


Epoch 2 finished, ave_loss: 0.0764, ave_loss_dict: {'diag': 0.015267550339407765, 'med': 0.09004634326498812, 'pro': 0.015465260615819314, 'lab': 0.06097104582029976, 'anomaly': 0.20001955103973}, perf_dict: {'diag': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'med': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'pro': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'lab': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}}


Epoch: 03, Step: 723, loss:0.0782, diag:0.0168, med:0.0986, pro:0.0150, lab:0.0858, anomaly:0.1750: 100%|█| 724/724 [01:32<00:00,  7.82it/s]


...

Epoch 31 finished, ave_loss: 0.0637, ave_loss_dict: {'diag': 0.013457522592481575, 'med': 0.08086530952225568, 'pro': 0.012684423473884881, 'lab': 0.055262570192224414, 'anomaly': 0.15613399985251505}, perf_dict: {'diag': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'med': {'precision': 0.017241379310344827, 'recall': 0.005977011494252874, 'f1': 0.008505747126436782}, 'pro': {'precision': 0.0012484394506866417, 'recall': 0.0006242197253433209, 'f1': 0.0008322929671244277}, 'lab': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}}


Epoch: 32, Step: 723, loss:0.0652, diag:0.0136, med:0.0922, pro:0.0120, lab:0.0570, anomaly:0.1512: 100%|█| 724/724 [01:34<00:00,  7.67it/s]


Epoch 32 finished, ave_loss: 0.0635, ave_loss_dict: {'diag': 0.013455058489180072, 'med': 0.08076611652105882, 'pro': 0.012703499805635017, 'lab': 0.05523606141751313, 'anomaly': 0.155122441275673}, perf_dict: {'diag': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'med': {'precision': 0.027586206896551724, 'recall': 0.009195402298850575, 'f1': 0.013563218390804599}, 'pro': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'lab': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}}


Epoch: 33, Step: 601, loss:0.0639, diag:0.0149, med:0.0702, pro:0.0152, lab:0.0599, anomaly:0.1593:  83%|▊| 602/724 [01:18<00:15,  7.72it/s]


KeyboardInterrupt: 

In [18]:
torch.save(model.cpu().state_dict(), f"{save_path}/pretrained_model.pt")